[사용자 정의 데이터셋 & 모델-학습]
- iris.csv -> 사용자 정의 데이터셋
- DNN 모델 -> 사용자 정의 모델

[1] 모듈 로딩 및 데이터 준비

In [80]:
#모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.classification import F1Score
from torchinfo import summary

from torch.utils.data import Dataset,DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [81]:
data_file='../data/iris.csv'

In [82]:
irisdf=pd.read_csv(data_file)
irisdf.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [83]:
#타겟 컬럼 수치화  w. LabeEncoder
encoder=LabelEncoder()
encoder.fit(irisdf['variety'])
irisdf['variety']=encoder.transform(irisdf['variety'])

[2] 사용자 정의 데이터셋 클래스 생성
- - -
- 클래스 목적: 학습용 데이터셋 텐서화 및 전처리
- 클래스 이름: custom_dataset
- 부모 클래스: torch.utils.data.Dataset
- 매개변수: featuredf,targetdf

In [84]:
class custom_dataset(Dataset):
    #데이터 로딩 및 전처리 진행, 인스턴스 생성 메서드
    def __init__(self,feature_df,target_df):

        #부모 클래스 생성
        super().__init__()
        self.feature_df=feature_df
        self.target_df=target_df
        
        self.n_rows=feature_df.shape[0]
        self.n_features=feature_df.shape[1]

    #데이터 개수 반환
    def __len__(self):
        return self.n_rows

    #특정 index의 데이터와 타겟 반환 메서드
    def __getitem__(self, idx):
        feature_ts=torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts=torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts,target_ts

In [85]:
#함수 기능: 파일 확장자별 데이터프레임 변환
#함수 이름: convertDF
#매개변수: data_path (파일 경로)
#함수 결과: DataFrame

def convertDF(data_path,exist_header=0):
    ext=data_path.rsplit('.')[-1]

    if ext=='csv':
        return pd.read_csv(data_path,header=exist_header)

    elif ext=='json':
        return pd.read_json(data_path,header=exist_header)
    
    elif ext in ['xlsx', 'xls']:
        return pd.read_excel(data_path,header=exist_header)
    
    else:
        return pd.read_table(data_path,header=exist_header)


In [86]:
#클래스 기능: 파일 기반의 데이터셋
#클래스 이름: file_dataset
#매개변수: data_path (파일 경로)
#부모 클래스: utils.data.Dataset

class file_dataset(Dataset):
    #데이터 로딩 및 전처리 진행, 인스턴스 생성 메서드
    def __init__(self,data_path):

        #부모 클래스 생성
        super().__init__()

        #데이터 파일 -> DataFrame
        data_df=convertDF(data_path)

        #DataFrame -> feature/target 추출
        self.feature_df=data_df[data_df.columns[:-1]]
        self.target_df=data_df[data_df.columns[-1:]]
        
        self.n_features=data_df.shape[1]
        self.n_rows=self.feature_df.shape[0]

    #데이터 개수 반환
    def __len__(self):
        return self.n_rows

    #특정 index의 데이터와 타겟 반환 메서드
    def __getitem__(self, idx):
        feature_ts=torch.FloatTensor(self.feature_df.iloc[idx].values)
        target_ts=torch.FloatTensor(self.target_df.iloc[idx].values)
        return feature_ts,target_ts

[3] 데이터셋 인스턴스 생성

In [87]:
irisdf.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [88]:
#feature/target 추출
feature_df, target_df=irisdf[irisdf.columns[:-1]],irisdf[[irisdf.columns[-1]]]
                                                # = irisdf[irisdf.columns[-1:]]                          
print(f'feature_df: {feature_df.shape}')
print(f'target_df: {target_df.shape}')

#iris 데이터셋 인스턴스 생성
irisds=custom_dataset(feature_df, target_df)

feature_df: (150, 4)
target_df: (150, 1)


In [89]:
#IRIS 데이터셋 속성
irisds.feature_df

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [90]:
irisds.n_features, irisds.n_rows

(4, 150)

In [91]:
#iris 데이터셋 메서드
irisds[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] DataLoader 인스턴스 생성
- dataset 인스턴스, 배치 사이즈 필요

In [92]:
irisdl=DataLoader(irisds,batch_size=32)     #batch_size=32가 성능이 젤 좋다

In [93]:
for data_ts,target_ts in irisdl:
    print(data_ts.shape,target_ts.shape)
    break

torch.Size([32, 4]) torch.Size([32, 1])


[5] 모델 준비
- 모델 이름: custom_model
- 부모 클래스: nn.Module
- 매개변수: None
- 모델 구조
    - 입력층: 입력  4개, 출력 10개, AF ReLU (for 기울기 소실 예방 -> 그래도 기울기 소실 문제 생기면  LeakyReLU)
    - 은닉층: 입력 10개, 출력 30개, AF ReLU
    - 출력층: 입력 30개, 출력  3개, AF SoftMax(다중 분류)

In [94]:
class custom_model(nn.Module):
    
    #모델 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4,10)
        self.hidden_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,3)

    #순방향 학습 메서드
    def forward(self,x):
        y=F.relu(self.in_layer(x))
        y=F.relu(self.hidden_layer(y))
        return self.out_layer(y)            #CrossEntropyLoss() 얘가 알아서 SoftMax() 실행함함

In [95]:
#모델 인스턴스 생성
model=custom_model()
print(model)
summary(model,input_size=(10000,4))

custom_model(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=3, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
custom_model                             [10000, 3]                --
├─Linear: 1-1                            [10000, 10]               50
├─Linear: 1-2                            [10000, 30]               330
├─Linear: 1-3                            [10000, 3]                93
Total params: 473
Trainable params: 473
Non-trainable params: 0
Total mult-adds (M): 4.73
Input size (MB): 0.16
Forward/backward pass size (MB): 3.44
Params size (MB): 0.00
Estimated Total Size (MB): 3.60

[6] 학습

In [100]:
ts_loss=[[],[]]
ts_score=[[],[]]

for epoch in range(10):        #epoch수만큼 반복
    loss_total=0
    score_total=0

    #배치 크기만큼 데이터와 타겟 추츨해서 학습 진행
    for data_ts,target_ts in irisdl:
        #배치 크기만큼의 학습 데이터
        print(data_ts.shape, target_ts.shape)
        target_ts=target_ts.reshape(-1).long()

        #배치 크기만큼 학습 진행
        pre_y=model(data_ts)
        print(pre_y.shape,target_ts.reshape(-1).shape)

        #손실 계산
        loss=nn.CrossEntropyLoss()(pre_y,target_ts.reshape(-1))
        #accuracy...?

        break

torch.Size([32, 4]) torch.Size([32, 1])
torch.Size([32, 3]) torch.Size([32])
